In [1]:
from Frame import Frame
from Camera import Camera
import scipy
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..','fly_model'))
sys.path.insert(0, parent_dir)
import numpy as np
import Plotter
from Skeleton import Skeleton
from Skin import Skin
import plotly.graph_objects as go
import Utils
import pandas as pd
%matplotlib qt

path = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov19_2022_03_03/'


frames = list(range(900,910,1))

image_name= []
for frame in frames:
    image_name += [f'P{frame}CAM{cam + 1}' for cam in range(4)]
frames = {f'{im_name}.jpg':Frame(path,im_name,idx) for idx,im_name in enumerate(image_name)}


points_3d = {body_wing : pd.DataFrame(Utils.load_hull(body_wing,path),columns = ['X','Y','Z','frame']) for body_wing in ['body','rwing','lwing']}
real_coord = scipy.io.loadmat(f'{path}/3d_pts/real_coord.mat')['all_coords']


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:


def get_hull_part(part):
    points_3d_frame = points_3d[part][points_3d[part]['frame'] == 900]
    real_coord_frame = real_coord[real_coord[:,3] == 900,:]
    points_in_ew_frame = np.array([real_coord_frame[points_3d_frame[ax] - 1,idx] for idx,ax in enumerate(['X','Y','Z'])]).T
    points_in_ew_frame_homo = np.hstack((points_in_ew_frame,np.ones([points_in_ew_frame.shape[0],1])))
    points_in_ew_frame  = np.column_stack((points_in_ew_frame,np.arange(1,points_in_ew_frame.shape[0] + 1)))
    return points_in_ew_frame

hull = [get_hull_part(part) for part in list(points_3d.keys())]

In [3]:
frame_number = 900 
frame_names = ['P900CAM1.jpg','P900CAM2.jpg','P900CAM3.jpg','P900CAM4.jpg']
frame_names = [f'P{frame_number}CAM{idx}.jpg' for idx in range(1,5)]

ray_world = np.vstack([frames[frame].camera_center_to_pixel_ray(frames[frame].cm) for frame in  frame_names])
camera_center = np.vstack([frames[frame].X0.T for frame in  frame_names])
point = Utils.triangulate_least_square(camera_center,ray_world)

rot_mat_ew_to_lab = frames['P900CAM1.jpg'].rotation_matrix_from_vectors(frames['P900CAM1.jpg'].R[2,:], [0,0,1])



In [17]:
from Frame import Frame
from Camera import Camera
import scipy
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..','fly_model'))
sys.path.insert(0, parent_dir)
import numpy as np
import Plotter
from Skeleton import Skeleton
from Skin import Skin
import plotly.graph_objects as go
import Utils
import pandas as pd
from Joint import Joint
from Bone import Bone
%matplotlib qt
path_to_mesh = 'G:/My Drive/Research/gaussian_splatting/mesh'
skin_parts = ['body','right_wing','left_wing']
scale = 1/1000
skin_translation = np.array([-0.1-1,0,1])*scale

# initilize skeletone, joints and bones
pitch_body = 0

root_no_bone = Joint([1,0,0],[0,-pitch_body,0],parent = None, end_joint_of_bone = False, scale = scale)
neck = Joint([0.6,0,0.3],[0,pitch_body,0],parent = root_no_bone, end_joint_of_bone = False, scale = scale)
neck_thorax =  Joint([0.6,0,0.3],[0,-25,0], parent = root_no_bone, end_joint_of_bone = False, scale = scale)
head  =Joint([0.3,0,0],[0,0,0], parent = neck, scale = scale)
thorax  =Joint([-1,0,0],[0,25,0], parent= neck_thorax ,scale = scale)
abdomen = Joint([-1.3,0,0],[0,0,0], parent = thorax, scale = scale)
right_sp_no_bone = Joint([0,0,0.3],[0,pitch_body,0],parent = root_no_bone, end_joint_of_bone = False , scale = scale, color = 'red')
right_wing_root = Joint([0,-0.3,0],[0,0,0], parent = right_sp_no_bone, end_joint_of_bone = False, scale = scale, color = 'red')
right_wing_tip = Joint([0,-2.2,0],[0,0,0], parent = right_wing_root, scale = scale, color = 'red')
left_sp_no_bone = Joint([0,0,0.3],[0,pitch_body,0], parent = root_no_bone, end_joint_of_bone = False, scale = scale, color = 'blue')
left_wing_root = Joint([0,0.3,0],[0,0,0],parent = left_sp_no_bone, end_joint_of_bone = False, scale = scale, color = 'blue')
left_wing_tip = Joint([0,2.2,0],[0,0,0], parent =left_wing_root, scale = scale, color = 'blue')

root_no_bone.assign_bones()


body = Skin(f'{path_to_mesh}/body.stl',scale = 1)
right_wing = Skin(f'{path_to_mesh}/right_wing.stl',scale = 1,constant_weight = right_wing_root.bone)
left_wing = Skin(f'{path_to_mesh}/left_wing.stl',scale = 1, constant_weight = left_wing_root.bone)

root_no_bone.calculate_weight(body.ptcloud_skin,body.constant_weight)
root_no_bone.calculate_weight(right_wing.ptcloud_skin,right_wing.constant_weight)
root_no_bone.calculate_weight(left_wing.ptcloud_skin,left_wing.constant_weight)


# skin.translate_ptcloud_skin(skin_translation)
# skin.calculate_weights(constant_weight = [None,'right_wing_root','left_wing_root'],th = 10)
# skeleton = Skeleton(parent_child,joints,bone_rotation,skin, scale = scale)




d:\Documents\gaussian_splat\model\fly_model\Skin.py:17: RuntimeWarning:

invalid value encountered in divide



[array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([1., 1., 1., ..., 1., 1., 1.]),
 array([0., 0., 0., ..., 0., 0., 0.])]

In [ ]:
def rotate_skin():

    if joint.bone:
        joint.rotate_to_new_position(weight[:,np.newaxis],points_homo)



In [9]:
points_homo = np.column_stack([self.ptcloud_skin,np.ones(self.ptcloud_skin.shape[0])])
normals_homo = np.column_stack([self.skin_normals,np.ones(self.skin_normals.shape[0])])

# for every bone, rotate the skin to the joint coordinate systm (in bind position), (each bone is defined by the coordinates of the parent joint)
# then rotate to the new position by multiplying by global_transformation (transforming from local joint coordinates to global) 
rotated_points = [skeleton.joints[joint_name].rotate_to_new_position(weight[:,np.newaxis],points_homo) for weight,joint_name in zip(self.weight.T,skeleton.bones)]
normals_rotated = [skeleton.joints[joint_name].rotate_normal_to_new_position(weight[:,np.newaxis],normals_homo) for weight,joint_name in zip(self.weight.T,skeleton.bones)]
normals_rotated = np.sum(normals_rotated,axis = 0)[:,0:3]
        
        

In [10]:
body = Skin(f'{path_to_mesh}/body.stl',scale = 1)
body.constant_weight

False

In [12]:
right_wing.constant_weight

In [11]:
root_no_bone.calculate_weight(body.ptcloud_skin,body.constant_weight)
root_no_bone.calculate_weight(right_wing.ptcloud_skin,right_wing.constant_weight)
root_no_bone.calculate_weight(left_wing.ptcloud_skin,left_wing.constant_weight)

[array([0.00221642, 0.00218968, 0.0022693 , ..., 0.00124042, 0.00124369,
        0.00124056]),
 array([0.00173751, 0.00170843, 0.00178861, ..., 0.00082024, 0.00081958,
        0.00082021]),
 array([0.00133452, 0.00131674, 0.00136502, ..., 0.00077603, 0.00077323,
        0.00077592]),
 array([0.00173571, 0.00172729, 0.00179698, ..., 0.00082675, 0.00082638,
        0.00082681]),
 array([0.00169315, 0.0016487 , 0.00173817, ..., 0.00067796, 0.00068209,
        0.00067807])]

In [6]:
right_wing.constant_weight

In [13]:
root_no_bone.calculate_weight(right_wing.ptcloud_skin,right_wing.constant_weight)

[array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([1., 1., 1., ..., 1., 1., 1.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.])]

In [16]:
root_no_bone.calculate_weight(left_wing.ptcloud_skin,left_wing.constant_weight)

[array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([1., 1., 1., ..., 1., 1., 1.]),
 array([0., 0., 0., ..., 0., 0., 0.])]

In [4]:
np.vstack(root_no_bone.calculate_weight(body.ptcloud_skin,visited = None)).T

array([[0.00221642, 0.00173751, 0.00133452, 0.00173571, 0.00169315],
       [0.00218968, 0.00170843, 0.00131674, 0.00172729, 0.0016487 ],
       [0.0022693 , 0.00178861, 0.00136502, 0.00179698, 0.00173817],
       ...,
       [0.00124042, 0.00082024, 0.00077603, 0.00082675, 0.00067796],
       [0.00124369, 0.00081958, 0.00077323, 0.00082638, 0.00068209],
       [0.00124056, 0.00082021, 0.00077592, 0.00082681, 0.00067807]])

In [48]:
def calculate_weight(joint,skin,visited = None):

    if visited is None:
        visited = set()  
        
    if joint in visited:
        return [] # on the way up, if reached an already visited parent, return an empty list
    
    visited.add(joint)
    
    # for each node on the way down add its wight to the list. 
    weight = [joint.bone.calculate_dist_from_bone(skin)] if joint.bone else []
        
    for child in joint.child:
        weight += (calculate_weight(child,skin,visited))
    return weight # while going up, return weight - thus adding all children eventually reaching a child we havvent visited yet, then call the function again to get the other branch
                # if al childrens are visited we will add them until we got to the root. 

np.vstack(calculate_weight(root_no_bone,body.ptcloud_skin,visited = None)).T

array([[0.00221642, 0.00173751, 0.00133452, 0.00173571, 0.00169315],
       [0.00218968, 0.00170843, 0.00131674, 0.00172729, 0.0016487 ],
       [0.0022693 , 0.00178861, 0.00136502, 0.00179698, 0.00173817],
       ...,
       [0.00124042, 0.00082024, 0.00077603, 0.00082675, 0.00067796],
       [0.00124369, 0.00081958, 0.00077323, 0.00082638, 0.00068209],
       [0.00124056, 0.00082021, 0.00077592, 0.00082681, 0.00067807]])

In [29]:
def calculate_weight(joint,skin,visited = None,weight = []):

    if visited is None:
        visited = set()  
        
    if joint not in visited:
        visited.add(joint)

    if joint.bone:
        weight.append(joint.bone.calculate_dist_from_bone(skin))

    for child in joint.child:
        calculate_weight(child,skin,visited,weight)
    return weight

print(calculate_weight(root_no_bone,body.ptcloud_skin,visited = None,weight = []))

[array([0.00221642, 0.00218968, 0.0022693 , ..., 0.00124042, 0.00124369,
       0.00124056]), array([0.00173751, 0.00170843, 0.00178861, ..., 0.00082024, 0.00081958,
       0.00082021]), array([0.00133452, 0.00131674, 0.00136502, ..., 0.00077603, 0.00077323,
       0.00077592]), array([0.00173571, 0.00172729, 0.00179698, ..., 0.00082675, 0.00082638,
       0.00082681]), array([0.00169315, 0.0016487 , 0.00173817, ..., 0.00067796, 0.00068209,
       0.00067807])]


In [ ]:
def calculate_weight(joint,skin,visited = None,weight = []):
    
    if visited is None:
        visited = set()  
    
    if joint not in visited:
        visited.add(joint)
    
        
    for child in joint.child:   
        if child.bone:
            weight.append(child.bone.calculate_dist_from_bone(skin))

        calculate_weight(child,body.ptcloud_skin)

calculate_weight(root_no_bone,body.ptcloud_skin)

[0.00221642 0.00218968 0.0022693  ... 0.00124042 0.00124369 0.00124056]
[0.00173751 0.00170843 0.00178861 ... 0.00082024 0.00081958 0.00082021]
[0.00133452 0.00131674 0.00136502 ... 0.00077603 0.00077323 0.00077592]
[0.00173571 0.00172729 0.00179698 ... 0.00082675 0.00082638 0.00082681]
[0.00169315 0.0016487  0.00173817 ... 0.00067796 0.00068209 0.00067807]


In [23]:
def calculate_weight(joint, skin, visited=None):
    if visited is None:
        visited = set()
        
    if joint in visited:
        return []  # Base case: if the joint has already been visited, return an empty list
    
    visited.add(joint)
    weights = []  # Local weight list for this recursive call

    for child in joint.child:
        if child.bone:
            # Add the weight for the current bone
            weights.append(child.bone.calculate_dist_from_bone(skin))
        
        # Recursively calculate weights for the child joints and accumulate
        weights.extend(calculate_weight(child, skin, visited))
        print(weights)

    return weights

# Example usage
weights = calculate_weight(root_no_bone, body.ptcloud_skin)
weights

[]
[array([0.00221642, 0.00218968, 0.0022693 , ..., 0.00124042, 0.00124369,
       0.00124056])]
[]
[array([0.00133452, 0.00131674, 0.00136502, ..., 0.00077603, 0.00077323,
       0.00077592])]
[array([0.00221642, 0.00218968, 0.0022693 , ..., 0.00124042, 0.00124369,
       0.00124056]), array([0.00173751, 0.00170843, 0.00178861, ..., 0.00082024, 0.00081958,
       0.00082021]), array([0.00133452, 0.00131674, 0.00136502, ..., 0.00077603, 0.00077323,
       0.00077592])]
[]
[array([0.00173571, 0.00172729, 0.00179698, ..., 0.00082675, 0.00082638,
       0.00082681])]
[array([0.00221642, 0.00218968, 0.0022693 , ..., 0.00124042, 0.00124369,
       0.00124056]), array([0.00173751, 0.00170843, 0.00178861, ..., 0.00082024, 0.00081958,
       0.00082021]), array([0.00133452, 0.00131674, 0.00136502, ..., 0.00077603, 0.00077323,
       0.00077592]), array([0.00173571, 0.00172729, 0.00179698, ..., 0.00082675, 0.00082638,
       0.00082681])]
[]
[array([0.00169315, 0.0016487 , 0.00173817, ..., 0.00

[array([0.00221642, 0.00218968, 0.0022693 , ..., 0.00124042, 0.00124369,
        0.00124056]),
 array([0.00173751, 0.00170843, 0.00178861, ..., 0.00082024, 0.00081958,
        0.00082021]),
 array([0.00133452, 0.00131674, 0.00136502, ..., 0.00077603, 0.00077323,
        0.00077592]),
 array([0.00173571, 0.00172729, 0.00179698, ..., 0.00082675, 0.00082638,
        0.00082681]),
 array([0.00169315, 0.0016487 , 0.00173817, ..., 0.00067796, 0.00068209,
        0.00067807])]

In [11]:
def calculate_weight(joint,visited = []):
    
    if joint.bone != None:
        print(joint.bone.calculate_dist_from_bone(body.ptcloud_skin))
    for joint in joint.child:
        if len(joint.child) == 0:
            return 

        calculate_weight(joint,visited)

calculate_weight(root_no_bone)

[0.00186642 0.00183742 0.00191938 ... 0.00087747 0.00087835 0.00087751]
[0.00186642 0.00183742 0.00191938 ... 0.00087747 0.00087835 0.00087751]
[0.00173751 0.00170843 0.00178861 ... 0.00082024 0.00081958 0.00082021]
[0.00168812 0.00166159 0.00174218 ... 0.00069395 0.00069575 0.00069404]
[0.00168812 0.00166159 0.00174218 ... 0.00069395 0.00069575 0.00069404]
[0.00168373 0.00164681 0.00173355 ... 0.0006684  0.0006717  0.00066851]
[0.00168373 0.00164681 0.00173355 ... 0.0006684  0.0006717  0.00066851]


In [14]:
left_wing_tip

In [4]:
left_wing_tip.parent

In [2]:
def update_child(joint):
    if joint.parent == None:
        return
    update_child(joint.parent)
    if joint not in joint.parent:
        joint.parent.child.append(joint)


update_child(left_wing_tip)

In [2]:
root_no_bone.child

In [4]:
left_sp_no_bone.child

In [ ]:

        if constant_weight in self.bones:
            weight_map = np.zeros((points.shape[0],len(self.bones)))
            weight_map[:,self.bones.index(constant_weight)] = 1
        else:
            weight = np.vstack([ self.joints[joint_name].bone.calculate_dist_from_bone(points[:,0:3], **kwargs) for joint_name in self.joints.keys() if self.joints[joint_name].bone is not None]).T
            weight_map = (1/weight)/np.sum((1/weight),1)[:,np.newaxis]
            weight_map[weight_map > th] = 1

In [5]:

# initilize skeletone, joints and bones
pitch_body = 40
new_translation =  {'root_no_bone':[1,0,0],
          'neck':[0.6,0,0.3],
          'neck_thorax': [0.6,0,0.3],
          'head':[0.3,0,0],
          'thorax':[-1,0,0],
          'abdomen':[-1.3,0,0] ,
          'right_sp_no_bone': [-0,0,0.3],
          'right_wing_root':[0,-0.3,0],
          'right_wing_tip':[0,-2.2,0],
          'left_sp_no_bone':[0,0,0.3],
          'left_wing_root':[0,0.3,0],
          'left_wing_tip':[0,2.2,0]
          }
new_translation = {joint_name: np.array(joint_translation)*scale  for joint_name,joint_translation in new_translation.items()}

new_rotation = {'root_no_bone':[0,-pitch_body,0],
          'neck':[0,pitch_body,0],
          'neck_thorax': [0,-25,0],
          'head':[0,0,0],
          'thorax':[0,25,0],
          'abdomen':[0,0,0] ,
          'right_sp_no_bone': [0,pitch_body,0],
          'right_wing_root':[0,0,0],
          'right_wing_tip':[0,0,0],
          'left_sp_no_bone':[0,pitch_body,0],
          'left_wing_root':[-0,0,0],
          'left_wing_tip':[0,0,0]
          }
ls_rotated = np.dot(rot_mat_ew_to_lab,point[:,np.newaxis]).T

new_translation['root_no_bone'] = ls_rotated[0] + new_translation['root_no_bone']
skeleton.update_skeleton(new_rotation,new_translation)
transformed_points, transformed_normals = skin.rotate_skin(skeleton)





In [6]:
import matplotlib.pyplot as plt
transformed_points_to_rotate = transformed_points - np.mean(transformed_points,axis = 0)
transformed_points_ew = np.dot(rot_mat_ew_to_lab.T,transformed_points.T).T
transformed_points_ew = transformed_points_ew 
frame_name = 'P900CAM4.jpg'
transformed_points_homo = frames[frame_name].homogenize_coordinate(transformed_points_ew)
projeced = frames[frame_name].project_on_image(transformed_points_homo)

# plt.imshow(frames[frame_name].image_no_bg)
# plt.scatter(projeced[:,0],projeced[:,1])

In [7]:


camera_center_rotated = np.dot(rot_mat_ew_to_lab,camera_center.T).T
ray_world_rotated = np.dot(rot_mat_ew_to_lab,ray_world.T).T
points_in_ew_frame_rotated = [np.dot(rot_mat_ew_to_lab,part[:,0:3].T).T for part in hull]


vector_dir = np.array(ray_world_rotated - camera_center_rotated )[:,0:3]
vector_dir_norm= (vector_dir.T/np.linalg.norm(vector_dir,axis = 1)).T



# new_point = np.dot(vector_dir[0][0:3],np.array([0,0,1])[:,np.newaxis]).T[0]
# scales = np.dot(vector_dir[0][0:3],np.array([0,0,1]))
ray_world2  = camera_center_rotated + vector_dir_norm*np.array([[0.3,0.22,0.22,0.22]]).T
camera_center2  = camera_center_rotated + vector_dir_norm*np.array([[0.26,0.16,0.16,0.16]]).T


In [8]:
fig = go.Figure()
fig = Plotter.plot_skeleton_and_skin_hull(transformed_points,skin,skeleton,opacity = 0.1,marker_dict_skeleton = {'size': 3},line_dict_skeleton ={'width': 3})
fig.show()

In [132]:
fig = go.Figure()
fig = Plotter.plot_skeleton_and_skin_hull(transformed_points,skin,skeleton,opacity = 0.1,marker_dict_skeleton = {'size': 3},line_dict_skeleton ={'width': 3})

marker_dict = {'size': 5, 'color': 'red'}
marker_dict_point = {'size': 5, 'color': 'blue'}

line_dict = {'width': 3, 'color': 'red'}
# fig = go.Figure()

marker_dict = {'size': 5, 'color': [[255,0,0],[0,255,0]]}


Plotter.scatter3d(fig,points_in_ew_frame_rotated[0],'hull_body', mode = 'markers',marker_dict =  {'color': 'blue'}) 
Plotter.scatter3d(fig,points_in_ew_frame_rotated[1],'hull_rwing', mode = 'markers',marker_dict = { 'color': 'magenta'}) 
Plotter.scatter3d(fig,points_in_ew_frame_rotated[2],'hull_lwing', mode = 'markers',marker_dict = {'color':'cyan'}) 

marker_dict_point = {'size': 5, 'color': 'magenta'}

Plotter.scatter3d(fig,ls_rotated,'least square', mode = 'markers',marker_dict = marker_dict_point) 

fig.show()

In [54]:
fig = go.Figure()
fig = Plotter.plot_skeleton_and_skin_hull(transformed_points,skin,skeleton,opacity = 0.1,marker_dict_skeleton = {'size': 3},line_dict_skeleton ={'width': 3})

marker_dict = {'size': 5, 'color': 'red'}
marker_dict_point = {'size': 5, 'color': 'blue'}

line_dict = {'width': 3, 'color': 'red'}
# fig = go.Figure()

marker_dict = {'size': 5, 'color': [[255,0,0],[0,255,0]]}

[Plotter.scatter3d(fig,np.vstack((origin,end)),idx, mode = 'markers',marker_dict = marker_dict) for origin,end,idx in zip(camera_center2, ray_world2,range(4))]
[Plotter.scatter3d(fig,np.vstack((origin,end)),idx, mode = 'lines',line_dict = line_dict) for origin,end,idx in zip(camera_center2, ray_world2,range(4))]

Plotter.scatter3d(fig,points_in_ew_frame_rotated[0],'hull_body', mode = 'markers',marker_dict =  {'color': 'blue'}) 
Plotter.scatter3d(fig,points_in_ew_frame_rotated[1],'hull_rwing', mode = 'markers',marker_dict = { 'color': 'magenta'}) 
Plotter.scatter3d(fig,points_in_ew_frame_rotated[2],'hull_lwing', mode = 'markers',marker_dict = {'color':'cyan'}) 

marker_dict_point = {'size': 5, 'color': 'magenta'}

Plotter.scatter3d(fig,ls_rotated,'least square', mode = 'markers',marker_dict = marker_dict_point) 

fig.show()

In [32]:

fig = go.Figure()
fig = Plotter.plot_skeleton_and_skin_hull(transformed_points,skin,skeleton,opacity = 0.1,marker_dict_skeleton = {'size': 3},line_dict_skeleton ={'width': 3})
fig.show()

In [34]:
ray_world2

array([[-0.00123098, -0.0229936 , -0.00810553],
       [ 0.00453605, -0.01595115,  0.00059876],
       [-0.009139  , -0.01799263,  0.00017689],
       [-0.00048583, -0.01106921, -0.01935558]])

In [129]:
fig = go.Figure()

marker_dict = {'size': 5, 'color': 'red'}
marker_dict_point = {'size': 5, 'color': 'blue'}

line_dict = {'width': 3, 'color': 'red'}
# fig = go.Figure()

[Plotter.scatter3d(fig,np.vstack((origin,end)),idx, mode = 'markers',marker_dict = marker_dict) for origin,end,idx in zip(camera_center, ray_world,range(4))]
[Plotter.scatter3d(fig,np.vstack((origin,end)),idx, mode = 'lines',line_dict = line_dict) for origin,end,idx in zip(camera_center, ray_world,range(4))]


marker_dict = {'size': 5, 'color': 'blue'}

[Plotter.scatter3d(fig,np.vstack((origin,end)),idx, mode = 'markers',marker_dict = marker_dict) for origin,end,idx in zip(camera_center2, ray_world2,range(4))]
[Plotter.scatter3d(fig,np.vstack((origin,end)),idx, mode = 'lines',line_dict = line_dict) for origin,end,idx in zip(camera_center2, ray_world2,range(4))]

fig.show()


In [182]:

transformed_points_to_rotate = transformed_points - np.mean(transformed_points,axis = 0)
transformed_points_ew = np.dot(rot_mat.T,transformed_points_to_rotate.T).T
transformed_points_ew = transformed_points_ew + np.mean(transformed_points,axis = 0)

transformed_points_homo = frames['P900CAM4.jpg'].homogenize_coordinate(transformed_points_ew)
projeced = frames['P900CAM4.jpg'].project_on_image(transformed_points_homo)

plt.imshow(frames['P900CAM4.jpg'].image_no_bg)
plt.scatter(projeced[:,1],projeced[:,0])

In [183]:

fig = go.Figure()
fig = Plotter.plot_skeleton_and_skin_hull(transformed_points_ew,skin,skeleton,opacity = 0.1,marker_dict_skeleton = {'size': 3},line_dict_skeleton ={'width': 3})

marker_dict = {'size': 5, 'color': 'red'}
marker_dict_point = {'size': 5, 'color': 'blue'}

line_dict = {'width': 3, 'color': 'red'}
# fig = go.Figure()

marker_dict = {'size': 5, 'color': [[255,0,0],[0,255,0]]}

[Plotter.scatter3d(fig,np.vstack((origin,end)),idx, mode = 'markers',marker_dict = marker_dict) for origin,end,idx in zip(camera_center2, ray_world2,range(4))]
[Plotter.scatter3d(fig,np.vstack((origin,end)),idx, mode = 'lines',line_dict = line_dict) for origin,end,idx in zip(camera_center2, ray_world2,range(4))]


Plotter.scatter3d(fig,point[:,np.newaxis].T,'least square', mode = 'markers',marker_dict = marker_dict_point) 

fig.show()